<a href="https://colab.research.google.com/github/Drfengze/ANN-CBIR/blob/main/prediction_on_colab_tf2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
# !pip3 install -q virtualenv
# !virtualenv tf2
# #!export LD_LIBRARY_PATH=/usr/local/cuda-11/lib64:/usr/local/cuda-11.2/lib64
# !source /content/tf2/bin/activate; 
!pip install -r /content/requirements.txt; pip list; 
!sudo ln -s /usr/local/cuda-11.2/lib64/libcusolver.so.11 /usr/local/cuda-11.2/lib64/libcusolver.so.10
!cd /usr/local/cuda-11.2/lib64;  cp libcusolver.so.11 /usr/local/lib/python3.7/dist-packages/tensorflow/python
!sudo ln -s /usr/local/lib/python3.7/dist-packages/tensorflow/python/libcusolver.so.11 /usr/local/lib/python3.7/dist-packages/tensorflow/python/libcusolver.so.10
# !source /content/tf2/bin/activate; python predict.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 10.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9 MB 65.1 MB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 1.4 MB 70.8 MB/s 
     |████████████████████████████████| 1.1 MB 77.3 MB/s 
     |████████████████████████████████| 13.3 MB 64.3 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 10.3 MB 69.6 MB/s 
     |████████████████████████████████| 14.8 MB 73.9 MB/s 
     |████████████████████████████████| 146 kB 48.9 MB/s 
     |████████████████████████████████| 40 kB 5.6 MB/s 
     |████████████████████████████████| 11.3 MB 46.6 MB/s 
     |███████████████████

In [9]:
import os
import segmentation_models as sm
import tensorflow.keras as K
import tensorflow_addons as tfa
import subprocess
import time
from pprint import pprint
import json
import xarray as xr
import rioxarray  # activate the rio accessor
from affine import Affine
import numpy as np
import tensorflow as tf
print(tf.__version__)
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
# from google.colab import drive


def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)


def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked


def get_file_list(FOLDER, image_base):
    filesList = subprocess.run(
        ['ls', FOLDER], stdout=subprocess.PIPE).stdout.decode("utf-8").splitlines()
    exportFilesList = [s for s in filesList if image_base in s]
    imageFilesList = []
    jsonFile = None
    for f in exportFilesList:
        if f.endswith('.tfrecord.gz'):
            imageFilesList.append(FOLDER+'/'+f)
        elif f.endswith('.json'):
            jsonFile = f
    imageFilesList.sort()
    return imageFilesList, jsonFile


def get_dataset(file_list):
    imageDataset = tf.data.TFRecordDataset(file_list, compression_type='GZIP')
    imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
    imageDataset = imageDataset.map(toTupleImage).batch(1)
    return imageDataset

# %%


def transfer(model, imageDataset):
    base_model = tf.keras.models.Model(
        inputs=model.inputs, outputs=model.layers[-2].output)
    base_model.trainable = False
    inputs = tf.keras.layers.Input(shape=[None, None, len(BANDS)])
    x = base_model(inputs, training=False)
    x = model.get_layer(index=-1)(x)
    m = tf.keras.models.Model(inputs, x)

    m.compile(
        optimizer=optimizers.get(OPTIMIZER),
        # loss=focal_tversky_loss,
        loss=sm.losses.DiceLoss(class_weights=[0.2, 0.8], ignore_index=-1),
        # loss = losses.get(LOSS),
        metrics=[metric for metric in METRICS])

    history = model.fit(imageDataset,
                        epochs=initial_epochs,
                        validation_data=validation_dataset)
    return m

# %%
def doPrediction(image_base, kernel_buffer, output_tf):
    """Perform inference on exported imagery, upload to Earth Engine.
    """
    print('Looking for TFRecord files...')
    imageFilesList, jsonFile = get_file_list(FOLDER, image_base)
    pprint(imageFilesList)
    jsonText = subprocess.run(
        ['cat', '/content/drive/MyDrive/New_South_Wales.json'], stdout=subprocess.PIPE).stdout.decode("utf-8")
    mixer = json.loads(jsonText)
    pprint(mixer)
    ls = np.append(
        np.arange(0, 70)*int(len(list(imageFilesList))/70), len(list(imageFilesList)))
    print(ls[:5], ls[-5:])
    # Perform inference.
    print('Running predictions...')
    # strat = tf.distribute.MirroredStrategy(devices=['/gpu:0', '/gpu:1'])
    print('Writing predictions...')
    writer = tf.io.TFRecordWriter(output_tf)
    patches = 0
    model = tf.keras.models.load_model(MODEL_DIR, compile=False, custom_objects={'sm_F1': sm.metrics.FScore(beta=1), "sm_F1_cls1": sm.metrics.FScore(
        beta=1, class_indexes=1), "sm_IoU": sm.metrics.IOUScore, 'precision': sm.metrics.Precision, 'recall': sm.metrics.Recall})

    if flag != 0:
        finetune_file_list = get_file_list(FOLDER, image_base, shape)
        finetune_imageDataset = get_dataset(
            finetune_file_list[ls[i]:ls[i+1]])
        m = transfer(model)
    else:
        m = model

    for i in range(len(ls) - 1):
        imageDataset = get_dataset(imageFilesList[ls[i]:ls[i+1]])
        predictions = m.predict(imageDataset, batch_size=1, verbose=1)
        labels = predictions.argmax(axis=3)
        for predictionPatch in labels:
            print('Writing patch ' + str(patches) + '...')
            predictionPatch = predictionPatch[
                x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]
            # Create an example.
            example = tf.train.Example(
                features=tf.train.Features(
                    feature={
                        'cropland': tf.train.Feature(
                            float_list=tf.train.FloatList(
                                value=predictionPatch.flatten()))
                    }
                )
            )
            # Write the example.
            writer.write(example.SerializeToString())
            patches += 1
    return output_tf

def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
        K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def jacard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true + y_pred)
    jac = (intersection + 1.) / (union - intersection + 1.)
    return K.mean(jac)


def to_image(prediction, mixer, saving_pre, raster_dir):
    f = open(mixer)
    mixer = json.load(f)
    print(mixer)
    f.close()
    doubleMatrix = mixer['projection']['affine']['doubleMatrix']
    patchesPerRow = mixer['patchesPerRow']

    def decode_fn(record_bytes):
        return tf.io.parse_single_example(
            # Data
            record_bytes,
            # Schema
            {'cropland': tf.io.FixedLenFeature([256*256], dtype=tf.float32)})
    result = tf.data.TFRecordDataset(prediction)
    ls = []
    for batch in result.map(decode_fn):
        ls.append(batch['cropland'].numpy().reshape([256, 256]))
    

    def slice_per(source, step):
        return [np.array(source[i::step]) for i in range(step)]
    ls = slice_per(ls, patchesPerRow)
    print('reshaping ...')
    ls = [np.concatenate(l) for l in ls]    
    ls = np.concatenate(ls,axis=1).astype('int8')
    print(saving_pre + '.npy')
    np.save(saving_pre + '.npy',ls)

    ds = xr.DataArray(ls)

    ds = ds.rio.write_crs(4326, inplace=True)
    transform = Affine(*doubleMatrix)
    ds.rio.write_transform(transform, inplace=True)
    ds.spatial_ref.GeoTransform
    ds.rio.set_spatial_dims('dim_1', 'dim_0')
    ds.rio.to_raster(
        raster_dir + saving_pre + ".tif")


2.4.0


In [10]:
with tf.device('/device:GPU:0'):
    print(tf.__version__)
    # %%
    YEAR = '2022'
    # FOLDER = 'data_aug/2020/Michigan2'
    MODEL_DIR = "/content/drive/MyDrive/RDA/model_and_env/AUS1/trainer/model3"
    BANDS = ['blue', 'green', 'red', 'nir', 'swir1',
             'swir2', 'ndvi', 'nirv', 'blue_1', 'green_1', 'red_1',
             'nir_1', 'swir1_1', 'swir2_1', 'ndvi_1', 'nirv_1']
    RESPONSE = 'cropland'
    FEATURES = BANDS + [RESPONSE]
    NCLASS = 2  

    # Specify the size and shape of patches expected by the model.
    KERNEL_SIZE = 256
    KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
    COLUMNS = [
        tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
    ]
    FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

    kernel_buffer = [128, 128]
    # Get set up for prediction.
    x_buffer = int(kernel_buffer[0] / 2)
    y_buffer = int(kernel_buffer[1] / 2)

    buffered_shape = [
        KERNEL_SHAPE[0] + kernel_buffer[0],
        KERNEL_SHAPE[1] + kernel_buffer[1]]

    imageColumns = [
        tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
        for k in BANDS
    ]
    imageFeaturesDict = dict(zip(BANDS, imageColumns))
    flag = 0

    states = ['New South Wales']
    state = states[0]
    print('predicting ')
    FOLDER = '/content/drive/MyDrive/RDA/prediciton/new/'
    image_base = 'training_us'
    tfname = '/content/{}.TFRecord'.format(state)

2.4.0
predicting 


In [11]:
tfname='/content/drive/MyDrive/New South Wales.TFRecord'

In [12]:
from google.colab import drive

In [13]:
# try:
#     doPrediction(image_base, kernel_buffer, tfname)
# except:
#     #   drive.mount('/content/drive', force_remount=True)
#     doPrediction(image_base, kernel_buffer, tfname)

json_file = '/content/drive/MyDrive/New_South_Wales.json'
tifname = state
raster_dir = '/content/drive/MyDrive/RDA/geotiff_result/'
try:
    # drive.mount('/content/drive', force_remount=True)
    ls = to_image(tfname, json_file, tifname, raster_dir)
except:
    drive.mount('/content/drive', force_remount=True)
    to_image(tfname, json_file, tifname, raster_dir)

{'projection': {'crs': 'EPSG:4326', 'affine': {'doubleMatrix': [0.00026949458523585647, 0.0, 140.9995669954001, 0.0, -0.00026949458523585647, -28.15706376002752]}}, 'patchDimensions': [256, 256], 'patchesPerRow': 262, 'totalPatches': 35370}
reshaping ...
New South Wales.npy


In [6]:
nsw = np.load('/content/New South Wales.npy')

In [13]:
import matplotlib.pyplot as plt